# Geo-Temporal Forecasting with Multivariate Graph Neural Network

Temporal Fusion Transformer (TFT) proposed by Lim et al. [1] is one of the most popular transformer-based model for time-series forecasting. In summary, TFT combines gating layers, an LSTM recurrent encoder, with multi-head attention layers for a multi-step forecasting strategy decoder. For more details on the Nixtla's TFT implementation visit [this link](https://nixtla.github.io/neuralforecast/models.tft.html).

In this notebook we show how to train the TFT model on the Texas electricity market load data (ERCOT). Accurately forecasting electricity markets is of great interest, as it is useful for planning distribution and consumption.

We will show you how to load the data, train the TFT performing automatic hyperparameter tuning, and produce forecasts. Then, we will show you how to perform multiple historical forecasts for cross validation.

You can run these experiments using GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/neuralforecast/blob/main/nbs/examples/Forecasting_TFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Libraries

In [ ]:
%%capture
!pip install neuralforecast

## 2. Load PEMS-BAY Data

The input to NeuralForecast is always a data frame in [long format](https://www.theanalysisfactor.com/wide-and-long-data/) with three columns: `unique_id`, `ds` and `y`:

* The `unique_id` (string, int or category) represents an identifier for the series. 

* The `ds` (datestamp or int) column should be either an integer indexing time or a datestamp ideally like YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp.

* The `y` (numeric) represents the measurement we wish to forecast. 
We will rename the 

First, read the 2022 historic total demand of the ERCOT market. We processed the original data (available [here](https://www.ercot.com/gridinfo/load/load_hist)), by adding the missing hour due to daylight saving time, parsing the date to datetime format, and filtering columns of interest.

In [ ]:
import pandas as pd

In [ ]:
Y_df = pd.read_csv('PEMS-BAY.csv')
#Y_df['ds'] = pd.to_datetime(Y_df['ds'])
#Y_df.head()

## 3. Model training and forecast

First, instantiate the `StemGNN` model.

To instantiate `AutoTFT` you need to define:

* `h`: forecasting horizon
* `loss`: training loss
* `config`: hyperparameter search space. If `None`, the `AutoTFT` class will use a pre-defined suggested hyperparameter space.
* `num_samples`: number of configurations explored.

In [ ]:
from ray import tune

from neuralforecast.models.stemgnn import StemGNN
from neuralforecast.core import NeuralForecast
from neuralforecast.losses.pytorch import MAE

import logging
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)

:::{.callout-tip}
XXX
:::

In [ ]:
horizon = 3
models = [StemGNN(h=horizon,
                  input_size=12,
                  n_series=XXX,
                  loss=MAE(),
                  config=None,
                  num_samples=3)]

:::{.callout-tip}
XXX
:::

The `NeuralForecast` class has built-in methods to simplify the forecasting pipelines, such as `fit`, `predit`, and `cross_validation`. Instantiate a `NeuralForecast` object with the following required parameters:

* `models`: a list of models.

* `freq`: a string indicating the frequency of the data. (See [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).)

Then, use the `fit` method to train the `AutoTFT` model on the ERCOT data. The total training time will depend on the hardware and the explored configurations, it should take between 10 and 30 minutes.

In [ ]:
%%capture
nf = NeuralForecast(
    models=models,
    freq='H')

nf.fit(df=Y_df)

Finally, use the `predict` method to forecast the next 24 hours after the training data and plot the forecasts.

In [ ]:
Y_hat_df = nf.predict()
Y_hat_df.head()

Plot the results with matplot lib

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize = (10, 3))
plot_df = pd.concat([Y_df.tail(24*5).reset_index(drop=True), Y_hat_df]).set_index('ds') # Concatenate the train and forecast dataframes
plot_df[['y', 'AutoTFT']].plot(ax=ax, linewidth=2)

ax.set_title('Load [MW]', fontsize=12)
ax.set_ylabel('Monthly Passengers', fontsize=12)
ax.set_xlabel('Date', fontsize=12)
ax.legend(prop={'size': 10})
ax.grid()

## 4. Cross validation for multiple historic forecasts

The `cross_validation` method allows you to simulate multiple historic forecasts, greatly simplifying pipelines by replacing for loops with `fit` and `predict` methods. See [this tutorial](https://nixtla.github.io/statsforecast/examples/getting_started_complete.html) for an animation of how the windows are defined. 

With time series data, cross validation is done by defining a sliding window across the historical data and predicting the period following it. This form of cross validation allows us to arrive at a better estimation of our model’s predictive abilities across a wider range of temporal instances while also keeping the data in the training set contiguous as is required by our models. The `cross_validation` method will use the validation set for hyperparameter selection, and will then produce the forecasts for the test set.

Use the `cross_validation` method to produce all the daily forecasts for September. Set the validation and test sizes. To produce daily forecasts set the forecasting set the step size between windows as 24, to only produce one forecast per day.

In [ ]:
%%capture
val_size  = 90*24 # 90 days x 24 hours
test_size = 30*24 # 30 days x 24 hours
fcst_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size,
                                n_windows=None, step_size=horizon)

Finally, we merge the forecasts with the `Y_df` dataset and plot the forecasts.

In [ ]:
Y_hat_df = fcst_df.reset_index(drop=True)
Y_hat_df = Y_hat_df.drop(columns=['y','cutoff'])

In [ ]:
plot_df = Y_df.merge(Y_hat_df, on=['unique_id','ds'], how='outer').tail(test_size+24*7)

plt.figure(figsize=(20,5))
plt.plot(plot_df['ds'], plot_df['y'], c='black', label='True')
plt.plot(plot_df['ds'], plot_df['AutoTFT'], c='blue', label='Forecast')
plt.axvline(pd.to_datetime('2022-09-01'), color='red', linestyle='-.')
plt.legend()
plt.grid()
plt.plot()

## Next Steps

In Challu et al [2] we demonstrate that the N-HiTS model outperforms the latest transformers by more than 20% with 50 times less computation.

Learn how to use the N-HiTS and the NeuralForecast library in [this tutorial](https://nixtla.github.io/neuralforecast/examples/electricitypeakforecasting.html).

## References

[1] [Lim, B., Arık, S. Ö., Loeff, N., & Pfister, T. (2021). Temporal fusion transformers for interpretable multi-horizon time series forecasting. International Journal of Forecasting, 37(4), 1748-1764.](https://www.sciencedirect.com/science/article/pii/S0169207021000637).

[2] [Cristian Challu, Kin G. Olivares, Boris N. Oreshkin, Federico Garza, Max Mergenthaler-Canseco, Artur Dubrawski (2021). N-HiTS: Neural Hierarchical Interpolation for Time Series Forecasting. Accepted at AAAI 2023.](https://arxiv.org/abs/2201.12886)